## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-01-20-51-05 +0000,nyt,"Live Updates: After a Weak Jobs Report, Trump ...",https://www.nytimes.com/live/2025/07/31/busine...
1,2025-08-01-20-49-44 +0000,nypost,Milwaukee sicko gets life in prison for killin...,https://nypost.com/2025/08/01/us-news/milwauke...
2,2025-08-01-20-48-38 +0000,nyt,How the Threat of Trump’s Highest Tariff Derai...,https://www.nytimes.com/2025/08/01/world/afric...
3,2025-08-01-20-48-00 +0000,wsj,Jury Says Tesla Must Pay $243 Million Over Fat...,https://www.wsj.com/business/autos/tesla-drive...
4,2025-08-01-20-47-00 +0000,wsj,A Weak Jobs Report and Revised Tariff Plans Se...,https://www.wsj.com/finance/investing/a-weak-j...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,81
131,tariffs,24
59,new,19
23,tariff,15
186,russian,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
261,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...,147
134,2025-08-01-15-15-26 +0000,wapo,Trump’s new tariffs will hit these major tradi...,https://www.washingtonpost.com/business/2025/0...,142
23,2025-08-01-20-18-00 +0000,wsj,Stock Market Today: Dow Falls on Tariff Blitz;...,https://www.wsj.com/livecoverage/jobs-report-t...,140
271,2025-07-31-23-55-00 +0000,wsj,President Trump is building a Mar-a-Lago style...,https://www.wsj.com/politics/policy/trump-ball...,140
246,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...,139


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
261,147,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...
272,62,2025-07-31-23-44-33 +0000,nypost,White House releases new tariff rates for doze...,https://nypost.com/2025/07/31/us-news/white-ho...
250,58,2025-08-01-02-17-15 +0000,nypost,Vile footage shows man slap woman for refusing...,https://nypost.com/2025/07/31/world-news/vile-...
36,49,2025-08-01-20-03-00 +0000,wsj,"U.S. job growth slowed in July, a signal that ...",https://www.wsj.com/economy/jobs/jobs-report-j...
246,48,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...
127,47,2025-08-01-15-34-42 +0000,nypost,"Dow drops over 500 points as Trump tariffs, jo...",https://nypost.com/2025/08/01/business/dow-dro...
15,41,2025-08-01-20-28-00 +0000,wsj,The Trump administration is blocking funding f...,https://www.wsj.com/politics/policy/trump-cdc-...
257,40,2025-08-01-01-10-11 +0000,bbc,Three top officials were asleep or out of town...,https://www.bbc.com/news/articles/czernwez4epo...
153,38,2025-08-01-14-08-00 +0000,wsj,"U.S. Envoy Steve Witkoff Visits Gaza as Trump,...",https://www.wsj.com/world/middle-east/u-s-envo...
221,36,2025-08-01-05-44-24 +0000,nypost,Kamala Harris complains ‘the system’ is ‘broke...,https://nypost.com/2025/08/01/us-news/kamala-h...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
